In [8]:
# %pip -q install datasets
# %pip -q install evaluate

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import torch
import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
import datasets
import evaluate
import numpy as np
from datasets import load_dataset
import pandas as pd

import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-20 18:07:01.521332: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appr

In [3]:
test = '/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv'
rulec_test = pd.read_csv(test, delimiter='\t')

In [4]:
# from datasets.dataset_dict import DatasetDict
from datasets import Dataset

test_ds = Dataset.from_dict({'corrupt_sent':rulec_test['corrupt_sent'],'correct_sent' : rulec_test['correct_sent']})

In [5]:
for i in test_ds.take(1):
    print(i)

{'corrupt_sent': 'Экологическая составляющая ситуации Аральского моря является в несколько видов .', 'correct_sent': 'Экологическая составляющая ситуации Аральского моря проявляется в нескольких аспектах .'}


In [ ]:
# model_name = 'mika5883/pretrain_rugec_msu' #prefix = 'grammar: '
model_name = 'mika5883/finetune_rugec_msu' #prefix = 'grammar: '
# name = 'Askinkaty/RuT5_GEC' #prefix = 'improve_grammar: ' or 'improve_grammar'
# model_name = 'mika5883/gan_gec'
# model_name = 'mika5883/MT5_large_A_art'
# name = 'mika5883/finetune_rugec_msu'
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
def tokenize_function(examples, prefix='grammar: '):
    # inputs = [f'grammar: {each}' for each in examples['corrupt_sent']]
    inputs = [f'{prefix}{each}' for each in examples]
    # inputs = [each for each in examples['corrupt_sent']]
    # targets = [each for each in examples['correct_sent']]

    model_inputs = tokenizer(inputs, max_length=128, padding='longest', truncation=True, return_tensors='pt')
    # labels = tokenizer(text_target=targets, max_length=128, padding='max_length', truncation=True, return_tensors='pt')
    # labels["input_ids"] = [
    #             [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    #         ] #замена токенов паддинга на -100, чтобы они не учитывались при подсчёте потерь
    # model_inputs["labels"] = labels["input_ids"]
    model_inputs['inputs'] = inputs
    return model_inputs

In [22]:
from transformers import Pipeline
from transformers.pipelines.pt_utils import KeyDataset

In [28]:
class MyPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "maybe_arg" in kwargs:
            preprocess_kwargs["maybe_arg"] = kwargs["maybe_arg"]
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs, maybe_arg=2):
        if isinstance(inputs, str):
            inputs = [inputs]
        model_inputs = tokenize_function(inputs).input_ids.to(self.device)
        return {"inputs": model_inputs}

    def _forward(self, model_inputs):
        with torch.no_grad():
            outputs = self.model.generate(
                **model_inputs, 
                max_new_tokens=128, 
                num_return_sequences=7, 
                num_beams=7
            )
        return outputs

    def postprocess(self, model_outputs):
        return tokenizer.batch_decode(
            model_outputs, 
            skip_special_tokens=True, 
            clean_up_tokenization_spaces=False
        )

    def __call__(self, inputs):
        return self.postprocess(self._forward(self.preprocess(inputs)))


In [29]:
gram = MyPipeline(model=model, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [11]:
corrected_sents = []
def div(outs=None, num_return_seqs:int=7) -> list[list[str]]:
    return [outs[i:i+num_return_seqs] for i in range(0, len(outs), num_return_seqs)]
# torch.cuda.empty_cache()

In [30]:
from math import ceil

corrected_sents = []
batch_size = 64
total_count = len(test_ds)
chunks = ceil(total_count / batch_size)

def div(outs, n=7):
    return [outs[i:i+n] for i in range(0, len(outs), n)]

for i in tqdm.tqdm(range(chunks), desc="Correcting"):
    batch = KeyDataset(test_ds, 'corrupt_sent')[i*batch_size:(i+1)*batch_size]
    outputs = gram(batch)
    corrected_sents.append(div(outputs))

Correcting: 100%|██████████| 79/79 [16:28<00:00, 12.51s/it]


In [31]:
len(corrected_sents)

79

In [20]:
len(corrected_sents[0])

64

In [32]:
cors = [b for a in corrected_sents for b in a]
len(cors)

5000

In [35]:
corrupt_corrected_pairs = [each for each in zip(KeyDataset(test_ds, 'corrupt_sent'), cors)]
# corrupt_corrected_pairs = [[each[0], each[1]] for each in corrupt_corrected_pairs]
corrupt_corrected_pairs[0]

('Экологическая составляющая ситуации Аральского моря является в несколько видов .',
 ['Экологическая составляющая ситуации Аральского моря является в несколько видов .',
  'Экологическая составляющая ситуации Аральского моря является несколько видов .',
  'Экологическая составляющая ситуации Аральского моря является всего несколько видов .',
  'Экологическая составляющая ситуации у Аральского моря является в несколько видов .',
  'Экологическая составляющая ситуации в Аральского моря является в несколько видов .',
  'Экологическая составляющая ситуации в Аральского моря является несколько видов .',
  'Экологическая составляющая ситуации у Аральского моря является несколько видов .'])

In [26]:
rulec_test.iloc[0]['correct_sent']

'Экологическая составляющая ситуации Аральского моря проявляется в нескольких аспектах .'

In [34]:
df = pd.DataFrame(corrupt_corrected_pairs, columns=['corrupt', 'hypotheses'])
# df.hypotheses = df.hypotheses.map(lambda x: [x])
df.head()

,corrupt,hypotheses
0,Экологическая составляющая ситуации Аральского...,[Экологическая составляющая ситуации Аральског...
1,"Смысл всегда выражается понятно , но термины и...","[Смысл всегда выражается понятно , но термины ..."
2,"Я продолжаю бродить по музею , и в какой-то мо...","[Я продолжаю бродить по музею , и в какой-то м..."
3,"Видали , как хорошо это получилась с ипотеками...","[Видали , как хорошо это получилось с ипотекам..."
4,"Со стороной этой красивой вещи , я поняла , чт...","[Со стороной этой красивой вещи , я поняла , ч..."


In [ ]:
df.to_csv('hypotheses_v1_mult.tsv', sep='\t', index=False)